In [46]:
# Hidden cell for imports
from dataclasses import dataclass
import pychor
import galois
from nacl.public import PrivateKey, PublicKey, SealedBox
import numpy as np
from typing import List
GF_2 = galois.GF(2)

p1 = pychor.Party('p1')
p2 = pychor.Party('p2')

from nacl.utils import random

# Protocol for 1-out-of-n OT
def protocol_ot(sender, receiver, inputs, selection, n):
    # Function for the Receiver to generate keys
    @pychor.local_function
    def gen_keys(selection, n):
        # Generate a single real key pair key = (sk, pk)
        key = PrivateKey.generate()
        public_keys = [PublicKey(random(PublicKey.SIZE)) for _ in range(n)]
        public_keys[selection] = key.public_key
        return key, public_keys

    # Function for the Sender to encrypt the secret inputs
    @pychor.local_function
    def encrypt_inputs(pub_keys, inputs):
        # Encode the inputs as bytes
        length = max([(int(x).bit_length() + 7) // 8 for x in inputs])
        inputs_bytes = [int(x).to_bytes(length, 'little') for x in inputs]
    
        # Encrypt the inputs
        encrypted_inputs = [SealedBox(pk).encrypt(x) for pk, x in \
                            zip(pub_keys, inputs_bytes)]
        return encrypted_inputs

    # Function for the Receiver to decrypt the result
    @pychor.local_function
    def decrypt_result(selection, key, encrypted_inputs):
        # Select the correct input
        selected_input = encrypted_inputs[selection]
        # Decrypt it and convert it from bytes to int
        plaintext = SealedBox(key).decrypt(selected_input)
        return int.from_bytes(plaintext, 'little')

    # Step 1: Generate keys and send to Sender
    sk, pub_keys = gen_keys(selection, n).untup(2)
    pub_keys.send(receiver, sender)

    # Step 2: Encrypt inputs and send to Receiver
    encrypted_inputs = encrypt_inputs(pub_keys, inputs)
    encrypted_inputs.send(sender, receiver)

    # Step 3: Decrypt result
    result = decrypt_result(selection, sk, encrypted_inputs)

    return result

# The GMW Protocol and Binary MPC

The Goldreich-Micali-Wigderson (GMW) protocol {cite}`goldreich1987play` is a classic MPC protocol typically described in terms of evaluating binary circuits. The GMW protocol performs multiplication using Oblivious Transfer (OT), in a similar way to the protocol for generating binary multiplication triples in Chapter 6 (if you squint, they're almost identical, except that GMW performs the OT during the  online phase of the protocol). The GMW protocol can be extended to support arithmetic circuits and more than 2 parties.

````{admonition} Further reading: Oblivious Transfer
:class: seealso

For more information on the GMW protocol, see **Section 3.2 of [Pragmatic MPC](https://securecomputation.org/)**.
````

The basic ideas behind the (basic) GMW protocol are:
- All values are bits (i.e. values in $GF(2)$)
- Parties hold additive secret shares of values, and shares are built as described in Chapter 2
- Parties perform addition locally, via the additive homomorphism of the shares
- Parties perform multiplication using OT

We first present the protocol for multiplication, then build a GMW-based `SecBit` class that implements the GMW protocol.

## GMW Multiplication

The GMW protocol uses additive shares of binary values. To perform the multiplication $z = x * y$, we'll need to compute $(x_1 + x_2)(y_1 + y_2)$ where $x_1$ is held by P1 and $x_2$ is held by P2. Since the values are all binary, we can take exactly the same approach as the multiplication triple generation protocol in Chapter 6: P1 will generate their share of the result $z_1$ randomly, then generate a truth table for the value of $z_2$ in terms of P2's shares of $x$ and $y$ and use OT to deliver $z_2$ to P2.

Since the function is the same as in the multiplication triple case, we re-use the function from Chapter 6.

In [40]:
@pychor.local_function
def truth_table(a1, b1, c1):
    possible_c2s = []
    # Consider all possibilities for a2 and b2
    for a2 in GF_2([0, 1]):
        for b2 in GF_2([0, 1]):
            # Compute the share c2 in terms of the others
            c2 = (a1+a2) * (b1+b2) - c1
            possible_c2s.append(c2)
    return possible_c2s

The protocol for multiplication in GMW proceeds as follows:

````{admonition} Protocol: GMW Multiplication
:class: note

The parties P1 and P2 follow the following steps:
1. P1 picks its share of the output, $z_1$, randomly
2. P1 generates the truth table for the value of $z_2$ using `truth_table` above
3. P1 and P2 run 1-out-of-4 OT. P1 acts as Sender and provides the truth table as secret inputs. P2 acts as Receiver and provides their shares $x_2$ and $y_2$ as selection bits

At the end of the protocol, P1 and P2 hold secret shares of $z$, such that $x * y = z$.
````

In [41]:
def protocol_gmw_mult(x, y):
    # Function for computing the selection index
    @pychor.local_function
    def compute_selection(a2, b2):
        return int(a2)*2 + int(b2)

    x1, x2 = x
    y1, y2 = y
    z1 = p1.constant(GF_2.Random())
    z2 = protocol_ot(sender=p1, receiver=p2, 
                     inputs=truth_table(x1, y1, z1),
                     selection=compute_selection(x2, y2),
                     n=4)
    return z1, pychor.locally(GF_2, z2)

In [42]:
with pychor.LocalBackend():
    # Example: 1*1 = 1
    x1 = p1.constant(GF_2(0))
    x2 = p2.constant(GF_2(1))
    y1 = p1.constant(GF_2(0))
    y2 = p2.constant(GF_2(1))

    z1, z2 = protocol_gmw_mult((x1, x2), (y1, y2))
    print('Result:', z1, z2)

Result: 1@{p1} 0@{p2}


## The GMW Protocol

Now we can use the multiplication protocol above to define `SecBit`, a class for secure bits, backed by the GMW protocol. We take exactly the same approach as `SecInt` from Chapter 5, but using GMW to perform the operations (rather than multiplication triples).

In [43]:
@pychor.local_function
def share(x):
    s1 = GF_2.Random()
    s2 = GF_2(x) - s1
    return s1, s2

@dataclass
class SecBit:
    # s1 is p1's share of the value, and s2 is p2's share
    # all values (and shares) are binary
    s1: GF_2
    s2: GF_2

    @classmethod
    def input(cls, val):
        """Secret share an input: p1 holds s1, and p2 holds s2"""
        s1, s2 = share(val).untup(2)
        if p1 in val.parties:
            s2.send(p1, p2)
            return SecBit(s1, s2)
        else:
            s1.send(p2, p1)
            return SecBit(s1, s2)

    def __add__(x, y):
        """Add two SecBit objects using local addition of shares"""
        return SecBit(x.s1 + y.s1, x.s2 + y.s2)

    def __mul__(x, y):
        """Multiply two SecBit objects using GMW multiplication"""
        r1, r2 = protocol_gmw_mult((x.s1, x.s2), (y.s1, y.s2))
        return SecBit(r1, r2)

    def reveal(self):
        """Reveal the secret value by broadcast and reconstruction"""
        self.s1.send(p1, p2)
        self.s2.send(p2, p1)
        return self.s1 + self.s2

Here's a simple example, demonstrating addition and multiplication of `SecBit`s.

In [44]:
with pychor.LocalBackend():
    x_input = p1.constant(1)
    y_input = p2.constant(0)

    # Create secret shares of the inputs
    x = SecBit.input(x_input)
    y = SecBit.input(y_input)

    # Online phase: compute (x+y)^3
    z = x + y
    result = z*z*z
    print('(x+y)^3:', result.reveal())

(x+y)^3: 1@{p1, p2}


## Applications of Binary MPC

Building applications with binary representations is a little more involved than with arithmetic ones, since most applications require more than just operations on a single bit. The most common approach is to adopt binary representations of larger numbers that mimic hardware, and then implement operations on those representations in exactly the same way as they are implemented in hardware - for example, an adder circuit for 32-bit integers.

We can define a `SecBitInt` class to represent integers using a *list of `SecBit` objects*, and define operations like addition and multiplication for this representation. The `input` class method takes an integer as input, and performs a bit decomposition to turn it into a binary representation before secret sharing the bits. The addition and multiplication methods call binary adder and multiplier functions that we'll define later. The `reveal` method reverses the process, turning the list of bits into a Python integer.

In [79]:
@pychor.local_function
def bit_decompose(x, nbits):
    x = int(x)
    return [(x >> i) & 1 for i in range(nbits)]

@pychor.local_function
def bit_compose(bits):
    return sum([int(b) * 2**i for i, b in enumerate(bits)])
    
@dataclass
class SecBitInt:
    bits: List[SecBit]

    @classmethod
    def input(cls, val, num_bits=32):
        bits = bit_decompose(val, num_bits).unlist(num_bits)
        sec_bits = [SecBit.input(b) for b in bits]
        return SecBitInt(sec_bits)

    def __add__(x, y):
        return SecBitInt(adder(x.bits, y.bits))

    def __mul__(x, y):
        return SecBitInt(multiplier(x.bits, y.bits))

    def reveal(self):
        revealed_bits = [b.reveal() for b in self.bits]
        return bit_compose(revealed_bits)

In [80]:
with pychor.LocalBackend():
    x = SecBitInt.input(p1.constant(3), num_bits=3)
    print('Representation of x:', x)
    print('Value of x:', x.reveal())

Representation of x: SecBitInt(bits=[SecBit(s1=0@{p1}, s2=1@{p1, p2}), SecBit(s1=0@{p1}, s2=1@{p1, p2}), SecBit(s1=0@{p1}, s2=0@{p1, p2})])
Value of x: 3@{p1, p2}


To define addition of `SecBitInt` objects, we can implement the same logic as an [adder circuit](https://en.wikipedia.org/wiki/Adder_(electronics)). This approach computes the result bitwise, maintaining a *carry bit* to track overflow from one digit to the next during the addition.

In [89]:
def adder(x, y):
    output = []
    carry = SecBit(p1.constant(GF_2(0)), p2.constant(GF_2(0)))
    for a, b in zip(x, y):
        out = a + b + carry
        output.append(out)
        carry = ((a + carry) * (b + carry) + carry)
    return output

In [90]:
with pychor.LocalBackend():
    x = SecBitInt.input(p1.constant(3), num_bits=3)
    print('x + x:', (x+x).reveal())

x + x: 6@{p1, p2}


If we run out of bits then the results will overflow, just like in hardware.

In [92]:
with pychor.LocalBackend():
    x = SecBitInt.input(p1.constant(3), num_bits=3)
    print('x + x + x:', (x+x+x).reveal())

x + x + x: 1@{p1, p2}


This approach of representing numbers using binary values and performing computations using approaches from hardware circuits is common in real MPC systems. The binary representation is flexible (more on this later), and operations in $GF(2)$ are cheap, so the performance of these systems often appears high when measured with microbenchmarks.

However, each conceptual operation (e.g. "add two numbers") requires potentially many binary operations, depending on the complexity of the circuit used to implement it. For example, our `adder` function performs one multiplication *per bit in the representation*, just like an actual adder circuit. This means that for 32-bit numbers, we need to perform 32 binary multiplications per integer addition - and each multiplication requires one OT. In systems based on arithmetic shares, integer addition is just field addition, and can be performed without any communication at all between the parties! As a result, arithmetic systems are often faster than binary ones, when the arithmetic representation is sufficient for the application.

